In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import numpy as np
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

sin = torch.sin
cos = torch.cos
exp = torch.exp
pi = torch.pi
torch.set_default_dtype(torch.float64)

epochs = 30000  # iterations
h = 100  # Test point grid density
N = 2000  # interior points
N1 = 100  # boundary points
N2 = 200  # data points

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)

setup_seed(123456)

def gen_testdata():
    data = np.load("/content/drive/MyDrive/2D_RS_Real_1_t.npz")
    t, x, y, usol = data["t"], data["x"], data["y"], data["u_sol"]
    xx, yy, tt = np.meshgrid(x, y, t)
    X = np.vstack((np.ravel(xx), np.ravel(yy), np.ravel(tt))).T
    z = usol.flatten()[:, None]
    return X, z

def l2_relative_error(z_true, z_pred):
    return np.linalg.norm(z_true - z_pred) / np.linalg.norm(z_true)

# Domain and Sampling
def interior(n=N):
    x = torch.rand(n, 1)*6 - 3
    y = torch.rand(n, 1)*6 - 3
    t = torch.rand(n, 1)
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def ini(n=N1):
    x = torch.rand(n, 1)*6 - 3
    y = torch.rand(n, 1)*6 - 3
    t = torch.zeros_like(x)
    cond = torch.where(x**2+y**2<0.25, torch.full_like(x,1), torch.zeros_like(x))
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def left(n=N1):
    t = torch.rand(n, 1)
    x = -3 * torch.ones_like(t)
    y = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def right(n=N1):
    t = torch.rand(n, 1)
    x = 3 * torch.ones_like(t)
    y = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def up(n=N1):
    t = torch.rand(n, 1)
    y = 3 * torch.ones_like(t)
    x = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def down(n=N1):
    t = torch.rand(n, 1)
    y = -3 * torch.ones_like(t)
    x = torch.rand(n, 1)*6 - 3
    cond = torch.zeros_like(t)
    return x.requires_grad_(True), y.requires_grad_(True), t.requires_grad_(True), cond

def data_interior(n=N2):
    X, Y = gen_testdata()
    ids = np.random.randint(X.shape[0], size=n)
    xy_t = X[ids]
    u_real = Y[ids]
    xy_t = torch.from_numpy(xy_t)
    cond = torch.from_numpy(u_real)
    xy_t.requires_grad_(True)
    return xy_t, cond

class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, 1),
        )

    def forward(self, x):
        features = self.net(x)
        u_val = torch.abs(features)
        #u_val = torch.clamp(features, min=0.0, max=5.0)
        u_bce = torch.sigmoid(u_val)
        return u_val, u_bce

loss = torch.nn.MSELoss()
bce_loss_fn = torch.nn.BCELoss()

def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                 create_graph=True,
                                 only_inputs=True)[0]
    else:
        return gradients(gradients(u, x), x, order=order-1)

def l_interior(u):
    x, y, t, cond = interior()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    u_t = gradients(u_val, t, 1)
    u_x = gradients(u_val, x, 1)
    u_y = gradients(u_val, y, 1)
    u_xx = gradients(u_val, x, 2)
    u_yy = gradients(u_val, y, 2)
    residual = u_t - 6*u_val*(u_x**2+u_y**2)-3*(u_val**2)*(u_xx + u_yy) - v*u_val
    return loss(residual, cond)

def l_ini(u):
    x, y, t, cond = ini()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_left(u):
    x, y, t, cond = left()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_right(u):
    x, y, t, cond = right()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_up(u):
    x, y, t, cond = up()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_down(u):
    x, y, t, cond = down()
    u_val, _ = u(torch.cat([x, y, t], dim=1))
    return loss(u_val, cond)

def l_data(u):
    xyt, cond = data_interior()
    _, u_bce = u(xyt)  # BCE
    cond_binary = cond.clamp(0, 1)
    return bce_loss_fn(u_bce, cond_binary)

# Test different initial values for v
initial_v_values = [1, 2, 3, 4, 5]
results = {}


for init_v in initial_v_values:
    print(f"\n=== Running with initial v = {init_v} ===")

    # Reset network and optimizer for each run
    u = MLP()
    v = torch.nn.Parameter(torch.tensor([init_v], dtype=torch.float64), requires_grad=True)
    opt = torch.optim.RAdam(list(u.parameters()) + [v], lr=1e-3)
    scheduler = StepLR(opt, step_size=1000, gamma=0.9)

    v_history = []
    error_history = []
    loss_history = []
    xy_t, u_real = gen_testdata()
    xy_t = torch.from_numpy(xy_t)

    for i in range(epochs):
        opt.zero_grad()

        # Calculate all loss components
        l_i = l_interior(u)
        l_d = l_data(u)
        l_initial = l_ini(u)
        l_u = l_up(u)
        l_dn = l_down(u)
        l_l = l_left(u)
        l_r = l_right(u)

        # Total loss with weighting
        l = l_i + l_initial + l_dn + l_u + l_l + l_r + 5*l_d

        # Backpropagation
        l.backward()
        torch.nn.utils.clip_grad_norm_(u.parameters(), 0.1)
        opt.step()
        scheduler.step()

        # Calculate relative error every epoch
        u_pred, _ = u(xy_t)
        error = l2_relative_error(u_real, u_pred.detach().numpy())

        # Record history
        v_history.append(v.item())
        error_history.append(error)
        loss_history.append(l.item())

        if (i+1) % 1000 == 0 or i == 0:
            print(f"Epoch {i+1}, L2 Relative Error: {error:.6f}, v: {v.item():.6f}, Loss: {l.item():.6f}")

    results[init_v] = {
        'v_history': v_history,
        'error_history': error_history,
        'loss_history': loss_history,
        'final_v': v.item(),
        'final_error': error_history[-1]
    }
    print(f"Final results for initial {init_v}: v={v.item():.6f}, Error={error_history[-1]:.6f}")


# Plot convergence for different initial v values
# Plot convergence for different initial v values
plt.figure(figsize=(10, 6))
for init_v, data in results.items():
    plt.plot(data['v_history'], label=f'Initial v={init_v}', linewidth=2)

plt.xlabel('Iterations', fontsize=14)
plt.ylabel('v Values', fontsize=14)
plt.title('Convergence of v with Different Initial Values', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)

# 获取所有 v 值范围
all_v_values = np.concatenate([data['v_history'] for data in results.values()])
v_min, v_max = all_v_values.min(), all_v_values.max()

# 上下稍微留一点空隙，并按 0.5 步长设置刻度
plt.ylim(v_min - 0.25, v_max + 0.25)
plt.gca().set_yticks(np.arange(np.floor(v_min - 0.25), np.ceil(v_max + 0.25) + 0.5, 0.5))

plt.show()


=== Running with initial v = 1 ===


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/2D_RS_Real_1_t.npz'